#Part 1

##Loading data

In [47]:
import urllib.request
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1FrEC8j4VcrQ8Od12M5HZOSNHDGLoivGV', "user-shows.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1Oghwz6LcxRblphATjA3Ela-1tuOgYWza', "shows.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1Jj_uDE8zHZqZiHOZFPqD5XnwOrz2Jwye', "graph-small.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1JJ78XF-93WWQC8OfdivPpoWzov4eU5FK', "graph-full.txt")


('graph-full.txt', <http.client.HTTPMessage at 0x7fa66a639e20>)

In [48]:
test = "1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 0 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0 1 0 0"
test_user_shows = [int(x) for x in test.split()]


In [49]:
%env PYTHONHASHSEED 3
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark

env: PYTHONHASHSEED=3


In [50]:
from math import sqrt
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import *
import numpy as np

spark = SparkSession.builder.master("local[*]").appName('SparkExample').config(
    "spark.executor.memory", "1g").config("spark.ui.port", "4050"
        ).getOrCreate()
sc = spark.sparkContext

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
      print(x) 

def to_list(x):
  """ A helper function to convert nested pyspark.resultiterable.ResultIterable iterables to lists """
  if hasattr(x, '__iter__'):
    if isinstance(x, pyspark.resultiterable.ResultIterable):
      return list(x)
    else:
      return [to_list(t) for t in x]
  else:
      return x

def isclose(x, y):
  """ A helper function to compare if two nested items are close numerically if they are floating point"""
  try:
    if hasattr(x, '__iter__'):
        return all(isclose(x[i], y[i]) for i in (x.keys() if isinstance(x, dict) else range(len(x))))
    else:
      if np.isreal(x):
        return np.isclose(x, y)
      else:
        return x == y
  except:
    return False

In [51]:

user_shows_file = sc.textFile("user-shows.txt")
user_shows_index = user_shows_file.zipWithIndex().map(lambda kv: (kv[1],kv[0]))
dbg(user_shows_index)
print(user_shows_index.count())

shows_file = sc.textFile("shows.txt")
shows_index = shows_file.zipWithIndex().map(lambda kv: (kv[1],kv[0]))
dbg(shows_index)
print(shows_index.count())

user_shows = user_shows_index.map(lambda kv: (kv[0], [int(x) for x in kv[1].split()]))
dbg(user_shows)

[(0, '1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

##1.1.a

In [52]:
from typing import List

def vector_mult(u: List[int], v: List[int]) -> int:
  # return sum(a+b for a, b in zip(u, v)) # No built-in functions so...
  assert len(u) == len(v)
  accum = 0
  for a, b in zip(u, v):
    accum += a*b
  return accum

def vector_len(v: List[int]) -> float:
  # Equivalent to math.hypot(*v)
  accum = 0
  for num in v:
    accum += num*num
  return sqrt(accum)

def cosine_similarity(u: List[int], v: List[int]) -> float:
    numerator = vector_mult(u, v)
    denominator = vector_len(u) * vector_len(v)
    return numerator / denominator

print(cosine_similarity([1,2,-1],[2,1,1]))
print(cosine_similarity([1,2,-1],[1,2,1]))
print(cosine_similarity([1,2,1],[1,2,1]))
print(cosine_similarity([1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 1, 1]))

0.5000000000000001
0.6666666666666667
1.0000000000000002
0.35355339059327373


In [53]:
assert isclose(cosine_similarity([1,2,-1],[2,1,1]), 0.5)
assert isclose(cosine_similarity([1,2,-1],[1,2,1]), 0.6666666666666667)
assert isclose(cosine_similarity([1,2,1],[1,2,1]), 1.0)
assert isclose(cosine_similarity([1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 1, 1]), 0.35355339059327373)


##1.1.b

In [54]:
from typing import Tuple
def q1b(test_user_shows: List[int]) -> pyspark.RDD[Tuple[int, float]]:
    # grab `user_shows` from the global namespace
    test_cosine_similarity = user_shows.map(lambda kv: (kv[0], cosine_similarity(test_user_shows, kv[1])))
    return test_cosine_similarity



    
print(q1b(test_user_shows).takeOrdered(5, lambda x: -x[1]))

[(6733, 0.5653963756883335), (1572, 0.5611939760061074), (2907, 0.5580880342929785), (5623, 0.5554706066152114), (2408, 0.5496782626513883)]


In [55]:
assert isclose([x[1] for x in q1b(test_user_shows).takeOrdered(5, lambda x: -x[1])], 
               [0.5653963756883335,0.5611939760061074,0.5580880342929785,0.5554706066152114,0.5496782626513883])

##1.1.c

In [56]:
def q1c(test_user_shows):
    user_id_to_cosine = q1b(test_user_shows)
    # Get the shows watched by the top 5
    top_5_users = user_id_to_cosine.takeOrdered(5, lambda kv: -kv[1])
    top_5_user_ids = [kv[0] for kv in top_5_users]
    top_5_user_to_shows_indices = user_shows.filter(lambda kv: kv[0] in top_5_user_ids)
    show_indices = top_5_user_to_shows_indices.map(lambda kv: kv[1])
    # Get the ones watched by all of them
    show_index_and_counts = (show_indices
                             .flatMap(enumerate)      # [1, 1, 0] => {0->1, 1->1, 2->0}
                             .filter(lambda kv: kv[1])# => {0->1, 1->1}
                             .map(lambda kv: (kv[0], 1))
                             .reduceByKey(lambda a, b: a+b)
    )
    shows_indices_everyone_watched = show_index_and_counts.filter(lambda kv: kv[1] == 5)
    # Compare to the test user
    test_user_shows_hasnt_watched_indices = {idx for idx, has_watched in enumerate(test_user_shows) if not has_watched}
    matching_show_indices = shows_indices_everyone_watched.filter(lambda kv: kv[0] in test_user_shows_hasnt_watched_indices)
    # Convert the indices back to string names
    matching_shows = shows_index.join(matching_show_indices).map(lambda kvv: kvv[1][0])
    return matching_shows



print(q1c(test_user_shows).collect())
# ['"Two and a Half Men"', '"The Oprah Winfrey Show"', '"The Early Show"', '"Family Guy"', '"George Lopez"', '"Today"', '"Everybody Loves Raymond"']

['"Today"', '"Everybody Loves Raymond"', '"Two and a Half Men"', '"The Oprah Winfrey Show"', '"The Early Show"', '"Family Guy"', '"George Lopez"']


In [57]:
assert set(q1c(test_user_shows).collect()) == set(['"Two and a Half Men"', '"The Oprah Winfrey Show"', '"The Early Show"', '"Family Guy"', '"George Lopez"', '"Today"', '"Everybody Loves Raymond"'])

##1.2.a

In [58]:
def q2a(filename) -> pyspark.RDD[Tuple[int, List[int]]]:
    raw_rdd = sc.textFile(filename)
    pairs = raw_rdd.map(lambda s_kv: tuple(int(n)-1 for n in s_kv.split()))
    return pairs.groupByKey()

dbg(to_list(q2a("graph-small.txt").sortByKey().take(3)))

[[0, [1, 7, 7, 13, 13, 21, 25, 43, 52, 65, 66, 77, 77, 78, 79, 81, 94]], [1, [2, 6, 12, 52, 52, 57, 60, 65, 89]], [2, [0, 3, 13, 19, 20, 34, 37, 47, 52, 52, 65, 71, 77, 82]]]


In [59]:
assert to_list(q2a("graph-small.txt").sortByKey().take(3)) == \
  [[0, [1, 7, 7, 13, 13, 21, 25, 43, 52, 65, 66, 77, 77, 78, 79, 81, 94]], 
   [1, [2, 6, 12, 52, 52, 57, 60, 65, 89]], 
   [2, [0, 3, 13, 19, 20, 34, 37, 47, 52, 52, 65, 71, 77, 82]]]


##1.2.b

In [60]:
from typing import Dict
from fractions import Fraction

def q2b_base_weights(vec: List[int]) -> Dict[int, float]:
    unique_items = set(vec)
    n = len(unique_items)
    return {item: 1/n for item in unique_items}

def q2b(graph_rdd: pyspark.RDD[Tuple[int, List[int]]]) -> pyspark.RDD[Tuple[int, Dict[int, float]]]:
    return graph_rdd.map(lambda kv: (kv[0], q2b_base_weights(kv[1])))

dbg(q2b(q2a("graph-small.txt")).sortByKey().take(3))

[(0, {1: 0.07142857142857142, 65: 0.07142857142857142, 66: 0.07142857142857142, 7: 0.07142857142857142, 43: 0.07142857142857142, 13: 0.07142857142857142, 77: 0.07142857142857142, 78: 0.07142857142857142, 79: 0.07142857142857142, 81: 0.07142857142857142, 52: 0.07142857142857142, 21: 0.07142857142857142, 25: 0.07142857142857142, 94: 0.07142857142857142}), (1, {89: 0.125, 65: 0.125, 2: 0.125, 6: 0.125, 12: 0.125, 52: 0.125, 57: 0.125, 60: 0.125}), (2, {0: 0.07692307692307693, 65: 0.07692307692307693, 34: 0.07692307692307693, 3: 0.07692307692307693, 37: 0.07692307692307693, 71: 0.07692307692307693, 13: 0.07692307692307693, 77: 0.07692307692307693, 47: 0.07692307692307693, 82: 0.07692307692307693, 19: 0.07692307692307693, 20: 0.07692307692307693, 52: 0.07692307692307693})]


In [61]:
assert isclose(q2b(q2a("graph-small.txt")).sortByKey().take(3),
               [(0, {1: 0.07142857142857142, 65: 0.07142857142857142, 66: 0.07142857142857142, 
                    7: 0.07142857142857142, 43: 0.07142857142857142, 13: 0.07142857142857142, 
                    77: 0.07142857142857142, 78: 0.07142857142857142, 79: 0.07142857142857142, 
                    81: 0.07142857142857142, 52: 0.07142857142857142, 21: 0.07142857142857142, 
                    25: 0.07142857142857142, 94: 0.07142857142857142}), 
                (1, {89: 0.125, 65: 0.125, 2: 0.125, 6: 0.125, 12: 0.125, 52: 0.125, 57: 0.125, 60: 0.125}),
                (2, {0: 0.07692307692307693, 65: 0.07692307692307693, 34: 0.07692307692307693, 
                    3: 0.07692307692307693, 37: 0.07692307692307693, 71: 0.07692307692307693, 
                    13: 0.07692307692307693, 77: 0.07692307692307693, 47: 0.07692307692307693, 
                    82: 0.07692307692307693, 19: 0.07692307692307693, 20: 0.07692307692307693, 
                    52: 0.07692307692307693})])

##1.2.c

In [62]:
from typing import Iterator

def sparse_cols_to_sparse_rows(c_rv: Tuple[int, Dict[int, float]]) -> Iterator[Tuple[int, Tuple[int, float]]]:
    col = c_rv[0]
    for row, val in c_rv[1].items():
      yield (row, (col, val))


def q2c(transition_matrix_col_rdd: pyspark.RDD[Tuple[int, Dict[int, float]]]):
    row_to_col_and_val = (transition_matrix_col_rdd
                          .sortByKey()  # Firstly we sort by column
                          .flatMap(sparse_cols_to_sparse_rows)
    )
    grouped_rows = (
        row_to_col_and_val
        .groupByKey() # Group into (row, [(col, val), ...])
        .sortByKey()  # Sort by row
    )
    return grouped_rows
    
    
dbg(to_list(q2c(q2b(q2a("graph-small.txt"))).map(lambda kv: (kv[0],sorted(kv[1]))).sortByKey().take(3)))

[[0, [[2, 0.07692307692307693], [10, 0.07142857142857142], [12, 0.1111111111111111], [13, 0.09090909090909091], [16, 0.08333333333333333], [23, 0.1111111111111111], [24, 0.08333333333333333], [27, 0.1111111111111111], [29, 0.1111111111111111], [30, 0.06666666666666667], [32, 0.08333333333333333], [37, 0.09090909090909091], [43, 0.1111111111111111], [45, 0.07692307692307693], [49, 0.09090909090909091], [50, 0.14285714285714285], [52, 0.25], [55, 0.1111111111111111], [57, 0.06666666666666667], [58, 0.058823529411764705], [63, 0.125], [64, 0.1111111111111111], [72, 0.125], [78, 0.07142857142857142], [81, 0.08333333333333333], [84, 0.1], [87, 0.125], [88, 0.08333333333333333], [93, 0.1111111111111111], [97, 0.16666666666666666], [99, 0.09090909090909091]]], [1, [[0, 0.07142857142857142], [25, 0.14285714285714285], [44, 0.0625], [57, 0.06666666666666667]]], [2, [[1, 0.125], [11, 0.1], [25, 0.14285714285714285], [66, 0.09090909090909091], [82, 0.06666666666666667], [85, 0.14285714285714285],

In [63]:
test_q2c = to_list(q2c(q2b(q2a("graph-small.txt"))).map(lambda kv: (kv[0],sorted(kv[1]))).sortByKey().take(3))
assert isclose(test_q2c,
  [[0, [[2, 0.07692307692307693], [10, 0.07142857142857142], [12, 0.1111111111111111], 
        [13, 0.09090909090909091], [16, 0.08333333333333333], [23, 0.1111111111111111], 
        [24, 0.08333333333333333], [27, 0.1111111111111111], [29, 0.1111111111111111], 
        [30, 0.06666666666666667], [32, 0.08333333333333333], [37, 0.09090909090909091], 
        [43, 0.1111111111111111], [45, 0.07692307692307693], [49, 0.09090909090909091], 
        [50, 0.14285714285714285], [52, 0.25], [55, 0.1111111111111111], [57, 0.06666666666666667], 
        [58, 0.058823529411764705], [63, 0.125], [64, 0.1111111111111111], [72, 0.125], 
        [78, 0.07142857142857142], [81, 0.08333333333333333], [84, 0.1], [87, 0.125], 
        [88, 0.08333333333333333], [93, 0.1111111111111111], [97, 0.16666666666666666], 
        [99, 0.09090909090909091]]], 
   [1, [[0, 0.07142857142857142], [25, 0.14285714285714285], [44, 0.0625], 
        [57, 0.06666666666666667]]],
   [2, [[1, 0.125], [11, 0.1], [25, 0.14285714285714285], [66, 0.09090909090909091], 
        [82, 0.06666666666666667], [85, 0.14285714285714285], [95, 0.09090909090909091]]]])


##1.2.d

In [64]:
def row_multiply(row, R):
    return sum(R[idx] * val for idx, val in row)

In [65]:
assert isclose(row_multiply([(0,1),(2,1)],[.2,.3,.5,0]), 0.7)
assert isclose(row_multiply([(1,1),(3,5)],[.2,.3,.5,0]), 0.3)
assert isclose(row_multiply([(0,.1),(1,.1),(2,.1),(3,.1)],[.2,.3,.5,0]), 0.1)

## 1.2.e
You don't need to write any code here, just run the below after passing all of the tests above.

In [66]:
def q2e(filename, num_iters=100):
  graph_rows = q2c(q2b(q2a(filename)))
  N = graph_rows.count()
  R = dict(enumerate([1/N]*N))
  row_results = graph_rows
  # R is technically not a vector; it is a dictionary of index to value
  for t in range(num_iters):
    # print(sorted(R.items()))
    vecR = sc.broadcast(R)
    #compute R again
    row_results = graph_rows.map(lambda kv: (kv[0], row_multiply(kv[1],vecR.value)))
    R = row_results.collectAsMap()
    
  print("R:",sorted(R.items()))
  return row_results.sortBy(lambda kv: -kv[1]).take(5)

# print(q2e("graph-small.txt"))

In [67]:
assert isclose(q2e("graph-small.txt"),
               [(52, 0.04243140406575616),
                (39, 0.039884267610139156),
                (13, 0.039377727003573254),
                (0, 0.03540059180244296),
                (26, 0.03489656776403212)])

[(0, 0.01), (1, 0.01), (2, 0.01), (3, 0.01), (4, 0.01), (5, 0.01), (6, 0.01), (7, 0.01), (8, 0.01), (9, 0.01), (10, 0.01), (11, 0.01), (12, 0.01), (13, 0.01), (14, 0.01), (15, 0.01), (16, 0.01), (17, 0.01), (18, 0.01), (19, 0.01), (20, 0.01), (21, 0.01), (22, 0.01), (23, 0.01), (24, 0.01), (25, 0.01), (26, 0.01), (27, 0.01), (28, 0.01), (29, 0.01), (30, 0.01), (31, 0.01), (32, 0.01), (33, 0.01), (34, 0.01), (35, 0.01), (36, 0.01), (37, 0.01), (38, 0.01), (39, 0.01), (40, 0.01), (41, 0.01), (42, 0.01), (43, 0.01), (44, 0.01), (45, 0.01), (46, 0.01), (47, 0.01), (48, 0.01), (49, 0.01), (50, 0.01), (51, 0.01), (52, 0.01), (53, 0.01), (54, 0.01), (55, 0.01), (56, 0.01), (57, 0.01), (58, 0.01), (59, 0.01), (60, 0.01), (61, 0.01), (62, 0.01), (63, 0.01), (64, 0.01), (65, 0.01), (66, 0.01), (67, 0.01), (68, 0.01), (69, 0.01), (70, 0.01), (71, 0.01), (72, 0.01), (73, 0.01), (74, 0.01), (75, 0.01), (76, 0.01), (77, 0.01), (78, 0.01), (79, 0.01), (80, 0.01), (81, 0.01), (82, 0.01), (83, 0.01), (

In [68]:
print(q2e("graph-full.txt"))


[(0, [(54, 0.07142857142857142), (100, 0.1), (167, 0.1), (226, 0.14285714285714285), (250, 0.1111111111111111), (397, 0.1111111111111111), (487, 0.14285714285714285), (685, 0.1), (893, 0.16666666666666666), (912, 0.2), (999, 0.07142857142857142)]), (1, [(0, 0.16666666666666666), (30, 0.1), (120, 0.1111111111111111), (441, 0.1111111111111111), (469, 0.08333333333333333), (583, 0.09090909090909091), (790, 0.2)]), (2, [(1, 0.09090909090909091), (58, 0.14285714285714285), (236, 0.14285714285714285), (459, 0.09090909090909091), (611, 0.125), (947, 0.125)]), (3, [(2, 0.1111111111111111), (311, 0.16666666666666666), (333, 0.125), (496, 0.16666666666666666), (591, 0.125), (655, 0.08333333333333333), (661, 0.1111111111111111), (676, 0.07142857142857142), (678, 0.1111111111111111), (712, 0.125), (747, 0.125), (798, 0.14285714285714285), (923, 0.16666666666666666)]), (4, [(3, 0.16666666666666666), (91, 0.14285714285714285), (275, 0.1), (293, 0.14285714285714285), (534, 0.16666666666666666), (669,

In [69]:
import urllib.request
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1Jj_uDE8zHZqZiHOZFPqD5XnwOrz2Jwye', "graph-small.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1JJ78XF-93WWQC8OfdivPpoWzov4eU5FK', "graph-full.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1bIaOO3SiLvim3gg5Dv7r7QcyxBLNuKkd', "facebook-medium.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1__W1Sc4-QWHYRcsGsryMG9ZbzNMN98IL', "facebook-large.txt")

('facebook-large.txt', <http.client.HTTPMessage at 0x7fa6943c8f40>)

In [70]:
%%writefile lab4_q5.txt
1 2
2 3
3 4
4 1
4 2
4 5
5 1
5 2
5 3
5 5

Overwriting lab4_q5.txt


In [72]:
# dbg(to_list(q2b(q2a("facebook-large.txt")).sortByKey().take(3)))
# print(q2e("facebook-large.txt"))
print(q2e("lab4_q5.txt", 0))
print(q2e("lab4_q5.txt", 1))


[(0, [(3, 0.3333333333333333), (4, 0.25)]), (1, [(0, 1.0), (3, 0.3333333333333333), (4, 0.25)]), (2, [(1, 1.0), (4, 0.25)]), (3, [(2, 1.0)]), (4, [(3, 0.3333333333333333), (4, 0.25)])]
R: [(0, 0.2), (1, 0.2), (2, 0.2), (3, 0.2), (4, 0.2)]


Py4JJavaError: ignored